In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import json

In [2]:
from matplotlib import rc
rc('font', family='sans-serif')
rc('font', size=13.0)
rc('text', usetex=False)
rc('figure', figsize=(11.69,8.27))

from matplotlib.font_manager import FontProperties

panel_label_font = FontProperties().copy()
panel_label_font.set_weight("bold")
panel_label_font.set_size(14.0)
panel_label_font.set_family("sans-serif")

# Seaborn styles
sns.set_style("whitegrid")

In [19]:
df = pd.read_csv("paperbuzz.csv", dtype={'status':str})
df.set_index("id", inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = pd.to_datetime(df['date'], errors = 'coerce')

df = df[~df.date.isnull()]

In [24]:
results = df.copy()

for ix, row in df.iterrows():
    if row['status'] == "200":
        j = json.loads(row['response'])
        if 'altmetrics_sources' in j:
            for s in j['altmetrics_sources']:
                name = s['source_id']
                if name in results.columns:
                    results.loc[ix, name] = int(s['events_count'])
                else:
                    results[name] = None
                    results.loc[ix, name] = int(s['events_count'])

In [27]:
results.describe()

,doi,date,status,response,timestamp,twitter,wikipedia,datacite
count,2349,2349,2349,2349,2349,65.0,1.0,2.0
unique,2349,38,3,2324,2349,17.0,1.0,2.0
top,10.11591/eei.v8i2.1493,2019-05-31 00:00:00,200,None,2019-07-09 10:16:38.432707,1.0,2.0,13.0
freq,1,645,2322,26,1,21.0,1.0,1.0
first,NaN,2019-05-31 00:00:00,NaN,NaN,2019-07-09 10:09:26.815129,NaN,NaN,NaN
last,NaN,2020-03-19 00:00:00,NaN,NaN,2019-07-09 10:16:57.349305,NaN,NaN,NaN
